In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv("train.csv")

In [2]:
df = df.fillna(0)

In [3]:
# create another column for timestamp (to not mess up the one used as index) and remove timezone info
df["timestamp"] = df['time']
df['timestamp'] = df['timestamp'].astype(str)
df['timestamp'] = df['timestamp'].map(lambda x: str(x)[:-6])
df['datetime'] = pd.to_datetime(df['timestamp'])

# create year, month, day, weekday, date, is_holiday, season (by full months only: Dec, Jan, Feb as winter)

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['weekday'] = df['datetime'].dt.weekday
df['hour'] = df['datetime'].dt.hour
df['date'] = df['datetime'].dt.date


# is_holiday
from datetime import date
import holidays
EE_holidays = holidays.EE()
df['is_holiday'] = [date in EE_holidays for date in df['date']]

# season
conditions = [
    (df['month'].isin(np.arange(3,6))), (df['month'].isin(np.arange(6,9))), 
    (df['month'].isin(np.arange(9,12))), (df['month'].isin(np.arange(1,3))), (df['month'] == 12)]
choices = ['Spring','Summer', 'Autumn', 'Winter', 'Winter']
df['season'] = np.select(conditions, choices)

# add google_trends info (weekly stats applied to all dates that week)
google_trends_weekly = [5,9,19,31,19,29,34,31,34,26,36,35,29,50,77,46,53,35,64,89,72,55,49,38,34,28,30,32,32,33,32,31,33,26,30,39,31,37,37,32,34,31,40,45,42,48,42,49,61,69,100,76]
google_trends_daily = np.repeat(google_trends_weekly,7)
index = [0,1,2]
google_trends_daily = np.delete(google_trends_daily, index)
google_trends_hourly = np.repeat(google_trends_daily,24)
df['google_trends_electricity'] = pd.Series(google_trends_hourly)


In [4]:
df = df.to_csv("df_clean.csv")